In [1]:
import pandas as pd
import glob


In [2]:
chunksize = 500_000
out_csv = "./data/warszawa_cell_rks.csv"
warszawa_cell_rks = set(pd.read_csv("./data/warszawa_cell_rks.csv")["cell_rk"])


In [4]:
cells = pd.read_csv("./data/hackplay_cells.gz")
cells_wawa = cells[cells["cell_rk"].isin(warszawa_cell_rks)]
cos = pd.read_csv("./data/hackplay_cos.gz")

In [9]:
files = sorted([
    *glob.glob("./data/user_locations_hackplay.gz"),
])

In [10]:
first_chunk = True
usecols = ["start_dttm", "cell_rk", "cos_rk", "user_id"]

In [11]:
for f in files:
    for chunk in pd.read_csv(
        f,
        chunksize=chunksize,
        usecols=usecols,
        dtype={"cell_rk": "int64", "cos_rk": "int64", "user_id": "string"},
        parse_dates=["start_dttm"],
    ):
        chunk = chunk[chunk["cell_rk"].isin(warszawa_cell_rks)]
        if chunk.empty:
            continue

        merged = (
            chunk
            .merge(cells_wawa, on="cell_rk", how="left")
            .merge(cos, on="cos_rk", how="left")
        )

        wanted = [
            "start_dttm", "user_id",
            "cell_rk", "lac", "cid", "technology", "frequency", "cell_lon", "cell_lat",
            "cos_rk", "cos_nm", "cos_family_nm",
        ]
        merged = merged[[c for c in wanted if c in merged.columns]]

        merged.to_csv(out_csv, index=False, mode="w" if first_chunk else "a", header=first_chunk)
        first_chunk = False

/var/folders/55/9xvhvhsj1cbf537pzbxyr4ww0000gn/T/ipykernel_56271/3635643216.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in pd.read_csv(
/var/folders/55/9xvhvhsj1cbf537pzbxyr4ww0000gn/T/ipykernel_56271/3635643216.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in pd.read_csv(
/var/folders/55/9xvhvhsj1cbf537pzbxyr4ww0000gn/T/ipykernel_56271/3635643216.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in pd.read_csv(


In [16]:
merged.to_csv("./data/hackplay_warszawa_full.csv", index=False)

In [17]:
pd_clean = pd.read_csv("./data/hackplay_warszawa_full.csv", parse_dates=["start_dttm"])

In [18]:
pd_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3080 entries, 0 to 3079
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   start_dttm     3079 non-null   datetime64[ns]
 1   user_id        3080 non-null   object        
 2   cell_rk        3080 non-null   int64         
 3   lac            3000 non-null   float64       
 4   cid            3080 non-null   float64       
 5   technology     3080 non-null   object        
 6   frequency      2986 non-null   float64       
 7   cell_lon       3080 non-null   float64       
 8   cell_lat       3080 non-null   float64       
 9   cos_rk         3080 non-null   int64         
 10  cos_nm         3080 non-null   object        
 11  cos_family_nm  3080 non-null   object        
dtypes: datetime64[ns](1), float64(5), int64(2), object(4)
memory usage: 288.9+ KB
